In [4]:
from datetime import datetime
import numpy as np
import pandas as pd
import googlemaps
from googleplaces import GooglePlaces, types, lang
import gmaps
import ipywidgets as widgets
import time
from datetime import date


### Configuring Google Places and Google Maps

In [6]:
# Open Google Places API key file and read keys
# The file is assumed to contain only one single line with the API key

def read_file_content(filename):
    """ Given a filename,
        returns the content of this file
    """
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

# Configure GooglePlaces and gmaps
PLACES_API_KEY = read_file_content("./API Keys/GOOGLE_PLACES_API_KEY.txt")
google_places = GooglePlaces(PLACES_API_KEY)
gmaps.configure(PLACES_API_KEY)


### Defining the Search Centers

We want to obtain all restaurants around Stuttgart, Germany. To overcome the limit of 20 (or 3x20 results when using a page token), we perform multiple search queries for different locations, and de-duplicate the results afterwards. So we specify an appropriate quadrangle within which the search centers are distributed with a given step size defining the distance between the search centers. 

In [7]:
def nearby_search(lat, lng, rankby='prominence', radius=None):
    '''
    Makes a nearby_search via python-google-places around the given lat and lng, with the given ranking criteria and search radius.
    Returns GooglePlacesSearchResult with up to 20 results.
    '''
    query_result = google_places.nearby_search(
        lat_lng={'lat': lat, 'lng': lng}, 
        rankby=rankby,
        radius=radius,
        types=[types.TYPE_RESTAURANT] or [types.TYPE_CAFE] or [type.TYPE_BAR] or [type.TYPE_CASINO])
    return query_result


In [8]:
# We define two rectangles within wich we perform are search queries:
# For the inner rectangle (Stuttgart city), we apply a smaller step width than for the outer rectangle (Stuttgart region).

# Define the corner points of rectangles as well as the step size. 

step = -0.029 # ~2km
smaller_step = step / 2

# Stuttgart area
start_lat = 48.85
end_lat = 48.66
start_long= 9.38
end_long = 8.95

# Stuttgart city
start_lat_city = 48.83
end_lat_city = 48.72
start_long_city = 9.25
end_long_city = 9.10


In [9]:
# Show search centers as markers on Google Maps

fig = gmaps.figure()

# Draw red markers for the wider area of Stuttgart
marker_locations=[]
for i in np.arange(start_lat, end_lat, step):
    for j in np.arange(start_long, end_long, step):
        # draw markers only if i and j is not within the city area
        if ((i > start_lat_city) | (i < end_lat_city) | (j > start_long_city) | (j < end_long_city)):
            marker_locations.append((i, j))
            
print("Number of red markers: {}".format(len(marker_locations)))   
markers_red = gmaps.symbol_layer(marker_locations, fill_color='red', stroke_color='red')
fig.add_layer(markers_red)

# Draw blue markers for the city area of Stuttgart
marker_locations=[]
for i in np.arange(start_lat_city, end_lat_city, smaller_step):
    for j in np.arange(start_long_city, end_long_city, smaller_step):
        marker_locations.append((i, j))

print("Number of blue markers: {}".format(len(marker_locations)))   
markers_blue = gmaps.symbol_layer(marker_locations, fill_color='blue', stroke_color='blue')
fig.add_layer(markers_blue)

fig


Number of red markers: 85
Number of blue markers: 88


Figure(layout=FigureLayout(height='420px'))

### Making the Google Places Queries and Building the Restaurant DataFrame

In [10]:
def make_google_places_nearby_search(marker_locations, rankby, radius):
    '''
    For each marker location, makes a nearby_request with the given search params.
    Returns a list of googleplaces.GooglePlacesSearchResult instances.
    '''
    query_results=[]
    for marker in marker_locations:
        
        # first page
        query_result = google_places.nearby_search(
            lat_lng={'lat': marker[0], 'lng': marker[1]},
            rankby=rankby,
            radius=radius,
            types=[types.TYPE_RESTAURANT] or [types.TYPE_CAFE] or [type.TYPE_BAR] or [type.TYPE_CASINO])
        query_results.append(query_result)
        
        # second and third page
        pagetoken = query_result.next_page_token
        while pagetoken:
            time.sleep(3) # wait, timing restiction at Google Places
            query_result_next_page = google_places.nearby_search(pagetoken=pagetoken)
            query_results.append(query_result_next_page)
            pagetoken = query_result_next_page.next_page_token
            if not pagetoken:
                break
    
    return query_results

query_results = make_google_places_nearby_search(marker_locations, 'prominence', 1500)


In [11]:
# For each query result store the id, name, geolocation and type in lists to create a DataFrame from
id_list, restaurant_list, lat_list, lng_list, types_list = [], [], [], [], []
for query_result in query_results:
    for place in query_result.places:
        id_list.append(place.place_id)
        restaurant_list.append(place.name)
        lat_list.append(float(place.geo_location['lat']))
        lng_list.append(float(place.geo_location['lng']))
        types_list.append(place.types)
        
assert(len(id_list) == len(restaurant_list) == len(lat_list) == len(lng_list) == len(types_list))
print("Number of restaurants: {}".format(len(restaurant_list)))
print("Number of uniqe restaurants: {}".format(len(set(restaurant_list))))

Number of restaurants: 3610
Number of uniqe restaurants: 1311


In [12]:
# Show restaurant locations on Google Maps

locations=[]
for i in range(len(lat_list)):
    locations.append([lat_list[i], lng_list[i]])
    
fig = gmaps.figure()
symbols = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red', scale=2)
fig.add_layer(symbols)
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create initial DataFrame
restaurant_df = pd.DataFrame(
    {'id': id_list,
     'name': restaurant_list,
     'latitude': lat_list,
     'longitude': lng_list,
     'types': types_list
    })

restaurant_df.head()

,id,name,latitude,longitude,types
0,ChIJdQ5X5XzPmUcRv7wL1Ond7Go,Hotel Hirsch,48.834179,9.263175,"[lodging, restaurant, food, point_of_interest,..."
1,ChIJCyT7XoDPmUcRVlr-BH_bNw8,"Restaurant ""La Perla"" im Schießsport-Zentrum",48.833400,9.249574,"[restaurant, food, point_of_interest, establis..."
2,ChIJleGdJZDJmUcRqozTDacuv14,Andys Crepes,48.831994,9.258656,"[restaurant, food, point_of_interest, store, e..."
3,ChIJjzY04XzFmUcRkoCkdy647Co,Restaurant TSV Steinhaldenfeld,48.829685,9.240347,"[restaurant, food, point_of_interest, establis..."
4,ChIJbc5V5XzPmUcRHvMxfGR2Afw,Oettingers Restaurant,48.834031,9.263453,"[restaurant, food, point_of_interest, establis..."


In [14]:
# Iterate over all types lists in restaurant_df.types 
# and produce one single set of types that occur in this restaurant_df
import itertools
set(list(itertools.chain.from_iterable([l for l in restaurant_df.types.values])))

{'amusement_park',
 'atm',
 'bakery',
 'bar',
 'book_store',
 'bowling_alley',
 'cafe',
 'car_repair',
 'car_wash',
 'casino',
 'church',
 'convenience_store',
 'establishment',
 'finance',
 'food',
 'gas_station',
 'grocery_or_supermarket',
 'gym',
 'hardware_store',
 'health',
 'home_goods_store',
 'liquor_store',
 'lodging',
 'meal_delivery',
 'meal_takeaway',
 'night_club',
 'park',
 'place_of_worship',
 'point_of_interest',
 'post_office',
 'restaurant',
 'school',
 'store',
 'supermarket'}

In [15]:
# Remove duplicates
restaurant_df.drop_duplicates(subset=['id'], inplace=True)

In [16]:
# To comply with the Google PLaces Terms & Conditions, we only save the restautant IDs 
restaurant_df_save = restaurant_df['id']

In [17]:
# Write to CSV file and Excel file
filename_long = str(date.today()) + '_google_restaurant_ids_step=' + str(step) + '_prominence=1500_pages=3'
filename_short = 'google_restaurant_ids'
restaurant_df_save.to_csv('./data/' + filename_long + '.csv', index=False)
restaurant_df_save.to_excel('./data/' + filename_long + '.xlsx', index=False)
restaurant_df_save.to_csv('./data/' + filename_short + '.csv', index=False)
restaurant_df_save.to_excel('./data/' + filename_short + '.xlsx', index=False)